In [1]:
from FuncionAzul import *

In [2]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "Test1/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "Test1/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "Test1/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "Test1/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "Test1/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "Test1/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "Test1/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = False, 
            Flag_csv = True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/03 13:50:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/08/03 13:51:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/08/03 13:51:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradat

22/08/03 13:51:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/03 13:51:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:51:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 1

...

22/08/03 13:52:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/03 13:52:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/03 13:52:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 13:52:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 1

/Users/arturo/opt/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
 13%|████▌                              | 19140/146275 [02:42<17:57, 118.04it/s]


KeyboardInterrupt: 

In [ ]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "Test1/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "Test1/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "Test1/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "Test1/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "Test1/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "Test1/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "Test1/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = True, 
            Flag_csv = True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/03 14:00:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/08/03 14:00:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:00:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:00:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:00:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:00:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/08/03 14:00:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradat

22/08/03 14:01:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/03 14:01:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 1

22/08/03 14:01:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 1

22/08/03 14:01:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/03 14:01:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/03 14:01:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 14:01:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 1

/Users/arturo/opt/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
 24%|████████▎                         | 35780/146275 [14:30<2:02:48, 15.00it/s]